# **Reconeixement de números amb IA**

In [ ]:
# 1. Instalar las librerías necesarias
!pip install tensorflow

# 2. Librerías necesarias
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import files

# 3. Cargar el dataset
# Para este ejemplo, supongamos que tienes un dataset organizado con carpetas para cada número del 1 al 11.

# Definir las rutas de los directorios
base_dir = '/content/numeros'  # Cambiar esta ruta si tienes tus datos en otro lugar
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# 4. Preprocesado de datos (normalización de imágenes)
train_datagen = ImageDataGenerator(rescale=1./255)  # Normaliza las imágenes entre 0 y 1
val_datagen = ImageDataGenerator(rescale=1./255)

# Crear generadores para entrenamiento y validación
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(100, 100),  # Redimensionamos las imágenes
    batch_size=32,
    class_mode='sparse',  # Usamos 'sparse' porque las etiquetas son enteros (1-11)
)

validation_generator = val_datagen.flow_from_directory(
    validation_dir,
    target_size=(100, 100),
    batch_size=32,
    class_mode='sparse',  # 'sparse' ya que estamos usando etiquetas enteras
)

# 5. Definición del modelo
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(100, 100, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(11, activation='softmax')  # 11 clases, una por cada número del 1 al 11
])

# 6. Compilar el modelo
model.compile(loss='sparse_categorical_crossentropy',  # Usamos 'sparse' porque las etiquetas son enteros
              optimizer='adam',
              metrics=['accuracy'])

# 7. Entrenamiento del modelo
history = model.fit(
    train_generator,
    epochs=10,  # Ajusta el número de épocas según sea necesario
    validation_data=validation_generator
)

# 8. Exportación del modelo en formato ligero
model_json = model.to_json()
with open("model_numeros.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights("model_numeros.weights.h5")

# 9. Descargar los archivos del modelo (JSON y pesos)
files.download("model_numeros.json")
files.download("model_numeros.weights.h5")

# 10. Gráfica de la evolución de la precisión
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

epochs = range(len(acc))
plt.plot(epochs, acc, 'r', label='Entrenamiento')
plt.plot(epochs, val_acc, 'b', label='Validación')
plt.title('Precisión del Modelo')
plt.legend()
plt.show()
